# Getting started with __htlogicalgates__

Welcome to `htlogicalgates`. This notebook shows you how to create hardware-tailored logical circuits for stabilizer codes using our package.

## Installation

The installation can conviniently be done using `pip`.

In [ ]:
%pip install htlogicalgates

Alternatively, you can clone the [repository](https://github.com/erkue/htlogicalgates) and include the files in your project. Finally, we can import the module:

In [2]:
import htlogicalgates as htlg

## How to tailor logical circuits

The main workflow for tailoring a circuit starts with creating three central objects:

In [3]:
stab_code = htlg.StabilizerCode("4_2_2")
conn = htlg.Connectivity("circular", num_qubits=4)
log_gate = htlg.Circuit("H 0", num_qubits=2)

After importing the package, we create three objects:

- `stab_code = StabilizerCode("4_2_2")`: First, we create a `StabilizerCode` object. It contains the stabilizer code for which we wish to find a logical circuit. As an argument we pass `"4_2_2"`, which indicates the $⟦4,2,2⟧$ color code. We can querry its stabilizers and logical Pauli operators and we can also create an object for our own stabilizer code.

- `conn = Connectivity("circular", num_qubits=4)`: Here, we create a `Connenctivity` object containing information about connections between qubits on the real hardware. Two-qubit gates are only allowed between connected qubits. Here, we use a `"circular"` connectivity on `num_qubits=4` qubits. There are multiple available connectivities and we can also create our own connectivity.

- `log_gate = Circuit("H 0", num_qubits=2)`: Finally, we create a `Circuit` object. We pass the string `"H 0"`, indicating the circuit, and the number of qubits its acts on `"num_qubits=2"`. Note that we count **qubits starting at 0**. This circuit indicates the logical action on the stabilizer code for which we tailor a circuit that implements this action on the logical level.

Note that we created a `Connectivity` for $n=4$ qubits and a logical `Circuit` for $k=2$ qubits since we are using the $⟦n=4,k=2,2⟧$-code.

Now, we can pass these objects to the function `tailor_logical_gate`:

In [4]:
circ, status = htlg.tailor_logical_gate(stab_code, conn, log_gate, num_cz_layers=2)

The function `tailor_logical_gate` takes the `StabilizerCode`, `Connectivity`, and `Circuit` object we created earlier as arguments. Furthermore, we indicate the number of controlled-Z gate layers we use in tailoring the circuit by using the argument `num_cz_layers`. Generally speaking, more controlled-Z layer make the ansatz more expressive and can lead to circuits with less two-qubit gates in total, while increasing runtime. If you can not find a specific gate, try to increase the number of controlled-Z gate layers.

The function returns a tuple: The return value `status` indicates the state of the optimization. There exist multiple status messages:

- `"Optimal"`: The returned circuit is optimal in terms of two-qubit gates.

- `"Bound {n}"`: The returned circuit is not optimal in terms of two-qubit games but there is no circuit with less than $n$ two-qubit gates.

- `"Infeasible"`: There is no physical circuit for the given stabilizer code, connectivity, logical circuit, and number of controlled-Z gate layer.

- `"Time out"`: A physical circuit was not found in the given time limit.

If the status message is `"Optimal"` or `"Bound {n}"`, the variable `circ` contains the physical circuit implementation. Otherwise, it is `None`.

In [5]:
print(f'Status: "{status}"')
print()
print("Circuit:")
print(circ)

Status: "Optimal"

Circuit:
Z 2
C_XYZ 0
CZ 0 1
CZ 1 2
C_XYZ 1
CZ 0 1
CZ 1 2
C_ZYX 0
S 1



## More ways to create a `StabilizerCode`

There exists a number of stabilizer codes that are already included in the package. A list of these can be accessed as follows:

In [6]:
htlg.available_stabilizercodes()

{'8_3_2': '[[8,3,2]] hypercube and color code.',
 '4_2_2': '[[4,2,2]] iceberg and color code.',
 '12_2_3': '[[12,2,3]] twisted toric-24 code.',
 'trivial': 'The trivial [[n,n,1]] code.'}

The keys of this dictionary, and sometimes an integer `n`, can be passed to the `StabilizerCode` constructor.

To create our own stabilizer codes, we can can create a list of logical Pauli-X, Pauli-Z and stabilizer operators. Hereby, Pauli operators are represented as strings and the correctness of the stabilizer code is checked when creating the stabilizer code object.

In [7]:
x_logicals = ["X0 X1 X2 X3 X4"]
z_logicals = ["Z0 Z1 Z2 Z3 Z4"]
stabilizers = ["X0 Z1 Z2 X3",
               "X1 Z2 Z3 X4",
               "X0 X2 Z3 Z4",
               "Z0 X1 X3 Z4"]

stab_code = htlg.StabilizerCode(x_logicals, z_logicals, stabilizers)

## More ways to create a `Connectivity`

Similarly to the `StabilizerCode` class, some baisc connectivities are included in the packaged. This can be queried as follows:

In [8]:
htlg.available_connectivities()

{'8_3_2_cube': 'Cube connectivity for the [[8,3,2]]-color code.',
 '12_2_3_checker': 'Checkerboard connectivity for the [[12,2,3]] twisted toric-24 code.',
 'linear': 'Connections exist between qubits $i$ and $i+1$.',
 'circular': 'Connections exist between qubits $i$ and $(i+1)%n$.',
 'all': 'Connections exist between all qubits.'}

The keys of this dictionary, and sometimes an integer `n`, can be passed to the `Connectivity` constructor.

Alternativly, we can create a `Connectivity` object by passing the an $n\times n$ adjacency matrix to the constructor.

In [9]:
import numpy as np

arr = np.array([[0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0]])
con = htlg.Connectivity(arr)

## More ways to create a `Circuit`

The interface of the circuit class is heavily inspired by other popular packages such as [qiskit](https://github.com/Qiskit/qiskit) or [stim](https://github.com/quantumlib/stim). A `Circuit` object can be created from a string or be build step-by-step. To create a circuit from a string, the string can be passed to the constructor:

In [10]:
string = """h 0
cz 0 1
h 1
c_xyz 0
"""
circ = htlg.Circuit(string)

Note that this string is not case-sensitive. The same circuit can also be build step-by-step:

In [11]:
circ = htlg.Circuit(2)
circ.h(0)
circ.cz(0, 1)
circ.h(1)
circ.c_xyz(0)

## Available gates

Next, we provide a list of all gates and their transformation of stabilizer generators.

- `'id'` gate: `X -> X, Z -> Z`

- `'x'` gate: `X -> X, Z -> -Z`

- `'y'` gate: `X -> -X, Z -> -Z`

- `'z'` gate: `X -> -X, Z -> Z`

- `'h'` gate: `X -> Z, Z -> X`

- `'s'` gate: `X -> Y, Z -> Z`

- `'sdg'` gate: `X -> -Y, Z -> Z`

- `'sxdg'` gate: `X -> X, Z -> Y`

- `'c_xyz'` gate: `X -> Y, Z -> X`

- `'c_zyx'` gate: `X -> Z, Z -> Y`

- `'swap'` gate: `XI -> IX, ZI -> IZ, IX -> XI, IZ -> ZI`

- `'cx'` gate: `XI -> XZ, ZI -> ZI, IX -> IX, IZ -> XZ`

- `'cz'` gate: `XI -> XZ, ZI -> ZI, IX -> ZX, IZ -> IZ`